# DualGAN model

> Defines the DualGAN model architecture.

In [1]:
# default_exp models.dualgan

In [2]:
#export
from fastai.vision.all import *
from fastai.basics import *
from typing import List
from fastai.vision.gan import *
from upit.models.cyclegan import *
from huggingface_hub import PyTorchModelHubMixin

In [3]:
#hide
from nbdev.showdoc import *

We use the models that were introduced in the [DualGAN paper](https://arxiv.org/abs/1704.02510). The original implementation is [here](https://github.com/duxingren14/DualGAN).

## Generator

In [4]:
#export
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [5]:
#export
class UNetDown(nn.Module):
    "Contracting layers of the Unet used in DualGAN"
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, stride=2, padding=1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_size, affine=True))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class UNetUp(nn.Module):
    "Expanding layers of the Unet used in DualGAN"
    def __init__(self, in_size, out_size, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, 4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(out_size, affine=True),
            nn.ReLU(inplace=True),
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)

        return x

In [6]:
show_doc(UNetDown,title_level=3)

<h3 id="UNetDown" class="doc_header"><code>class</code> <code>UNetDown</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>UNetDown</code>(**`in_size`**, **`out_size`**, **`normalize`**=*`True`*, **`dropout`**=*`0.0`*) :: `Module`

Contracting layers of the Unet used in DualGAN

In [10]:
#export
class DualGANGenerator(nn.Module):
    "Generator model for the DualGAN"
    def __init__(self, channels=3):
        super(DualGANGenerator, self).__init__()

        self.down1 = UNetDown(channels, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5, normalize=False)

        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 256)
        self.up5 = UNetUp(512, 128)
        self.up6 = UNetUp(256, 64)

        self.final = nn.Sequential(nn.ConvTranspose2d(128, channels, 4, stride=2, padding=1), nn.Tanh())

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        u1 = self.up1(d7, d6)
        u2 = self.up2(u1, d5)
        u3 = self.up3(u2, d4)
        u4 = self.up4(u3, d3)
        u5 = self.up5(u4, d2)
        u6 = self.up6(u5, d1)

        return self.final(u6)

In [11]:
show_doc(DualGANGenerator,title_level=3)

<h3 id="DualGANGenerator" class="doc_header"><code>class</code> <code>DualGANGenerator</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>DualGANGenerator</code>(**`channels`**=*`3`*) :: `Module`

Generator model for the DualGAN

### Test generator
Let's test for a few things:
1. The generator can indeed be initialized correctly
2. A random image can be passed into the model successfully with the correct size output

First let's create a random batch:

In [12]:
img1 = torch.randn(4,3,256,256)

In [13]:
m = DualGANGenerator(3)
with torch.no_grad():
    out1 = m(img1)
out1.shape

torch.Size([4, 3, 256, 256])

In [14]:
test_eq(out1.shape, torch.Size([4, 3, 256, 256]))

## Discriminator

As described in the DualGAN paper, we will use a 70x70 PatchGAN, the same discriminator for the CycleGAN.

In [15]:
D = discriminator(3)
print(D)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (4): LeakyReLU(negative_slope=0.2, inplace=True)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (7): LeakyReLU(negative_slope=0.2, inplace=True)
  (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (10): LeakyReLU(negative_slope=0.2, inplace=True)
  (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
)


In [17]:
#export
class DualGAN(nn.Module, PyTorchModelHubMixin):
    """
    DualGAN model. \n
    When called, takes in input batch of real images from both domains and outputs fake images for the opposite domains (with the generators). 

    Attributes: \n
    `G_A` (`nn.Module`): takes real input B and generates fake input A \n
    `G_B` (`nn.Module`): takes real input A and generates fake input B \n
    `D_A` (`nn.Module`): trained to make the difference between real input A and fake input A \n
    `D_B` (`nn.Module`): trained to make the difference between real input B and fake input B \n
    """
    def __init__(self, ch_in:int=3, n_features:int=64, disc_layers:int=3, lsgan:bool=False, 
                 drop:float=0., norm_layer:nn.Module=None):
        """
        Constructor for DualGAN model.
        
        Arguments: \n
        `ch_in` (`int`): Number of input channels (default=3) \n
        `n_features` (`int`): Number of input features (default=64) \n
        `disc_layers` (`int`): Number of discriminator layers (default=3) \n
        `lsgan` (`bool`): LSGAN training objective (output unnormalized float) or not? (default=True) \n
        `norm_layer` (`nn.Module`): Type of normalization layer to use in the models (default=None)
        """
        
        super().__init__()
        #G_A: takes real input B and generates fake input A
        #G_B: takes real input A and generates fake input B
        #D_A: trained to make the difference between real input A and fake input A
        #D_B: trained to make the difference between real input B and fake input B
        self.D_A = discriminator(ch_in, n_features, disc_layers, norm_layer, sigmoid=not lsgan)
        self.D_B = discriminator(ch_in, n_features, disc_layers, norm_layer, sigmoid=not lsgan)
        self.G_A = DualGANGenerator(ch_in); self.G_A.apply(weights_init_normal)
        self.G_B = DualGANGenerator(ch_in); self.G_B.apply(weights_init_normal)
        
    
    def forward(self, input):
        """Forward function for DualGAN model. The input is a tuple of a batch of real images from both domains A and B."""
        real_A, real_B = input
        fake_A, fake_B = self.G_A(real_B), self.G_B(real_A)
        idt_A, idt_B = self.G_A(real_A), self.G_B(real_B) #optional identity loss
        return [fake_A, fake_B, idt_A, idt_B]

In [18]:
show_doc(DualGAN,title_level=3)

<h3 id="DualGAN" class="doc_header"><code>class</code> <code>DualGAN</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>DualGAN</code>(**`ch_in`**:`int`=*`3`*, **`n_features`**:`int`=*`64`*, **`disc_layers`**:`int`=*`3`*, **`lsgan`**:`bool`=*`False`*, **`drop`**:`float`=*`0.0`*, **`norm_layer`**:`Module`=*`None`*) :: `Module`

DualGAN model. 

When called, takes in input batch of real images from both domains and outputs fake images for the opposite domains (with the generators). 

Attributes: 

`G_A` (`nn.Module`): takes real input B and generates fake input A 

`G_B` (`nn.Module`): takes real input A and generates fake input B 

`D_A` (`nn.Module`): trained to make the difference between real input A and fake input A 

`D_B` (`nn.Module`): trained to make the difference between real input B and fake input B 

In [19]:
show_doc(DualGAN.__init__)

<h4 id="DualGAN.__init__" class="doc_header"><code>DualGAN.__init__</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>DualGAN.__init__</code>(**`ch_in`**:`int`=*`3`*, **`n_features`**:`int`=*`64`*, **`disc_layers`**:`int`=*`3`*, **`lsgan`**:`bool`=*`False`*, **`drop`**:`float`=*`0.0`*, **`norm_layer`**:`Module`=*`None`*)

Constructor for DualGAN model.

Arguments: 

`ch_in` (`int`): Number of input channels (default=3) 

`n_features` (`int`): Number of input features (default=64) 

`disc_layers` (`int`): Number of discriminator layers (default=3) 

`lsgan` (`bool`): LSGAN training objective (output unnormalized float) or not? (default=True) 

`norm_layer` (`nn.Module`): Type of normalization layer to use in the models (default=None)

In [20]:
show_doc(DualGAN.forward)

<h4 id="DualGAN.forward" class="doc_header"><code>DualGAN.forward</code><a href="__main__.py#L37" class="source_link" style="float:right">[source]</a></h4>

> <code>DualGAN.forward</code>(**`input`**)

Forward function for DualGAN model. The input is a tuple of a batch of real images from both domains A and B.

### Quick model tests

Again, let's check that the model can be called sucsessfully and outputs the correct shapes.


In [21]:
dualgan_model = DualGAN()
img1 = torch.randn(4,3,256,256)
img2 = torch.randn(4,3,256,256)

In [22]:
%%time
with torch.no_grad(): dualgan_output = dualgan_model((img1,img2))

CPU times: user 18.9 s, sys: 1.57 s, total: 20.5 s
Wall time: 447 ms


In [23]:
test_eq(len(dualgan_output),4)
for output_batch in dualgan_output:
    test_eq(output_batch.shape,img1.shape)

In [27]:
#skip
dualgan_model.push_to_hub('upit-dualgan-test')

Cloning https://huggingface.co/tmabraham/upit-dualgan-test into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/340M [00:00<?, ?B/s]

To https://huggingface.co/tmabraham/upit-dualgan-test
   dccaa0f..f8d92db  main -> main



'https://huggingface.co/tmabraham/upit-dualgan-test/commit/f8d92db7854429ca64335e9ab698d7e7f2f44feb'

In [30]:
#skip
#hide_output
dualgan_model.from_pretrained('tmabraham/upit-dualgan-test')

config.json not found in HuggingFace Hub


Downloading:   0%|          | 0.00/357M [00:00<?, ?B/s]

DualGAN(
  (D_A): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (12): Sigmoid()
  )
  (D_B): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stri

In [31]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_models.cyclegan.ipynb.
Converted 01b_models.junyanz.ipynb.
Converted 02_data.unpaired.ipynb.
Converted 03_train.cyclegan.ipynb.
Converted 04_inference.cyclegan.ipynb.
Converted 05_metrics.ipynb.
Converted 06_tracking.wandb.ipynb.
Converted 07_models.dualgan.ipynb.
Converted 08_train.dualgan.ipynb.
Converted 09_models.ganilla.ipynb.
Converted index.ipynb.
